Get url list of user

In [1]:
from json import loads
import argparse
from platform import system
from re import match
from os import environ
from os.path import expanduser

In [2]:
filter_name_list = {}
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&#39;",
    ">": "&gt;",
    "<": "&lt;",
}
if system() == "Darwin":
    input_filename = expanduser("~/Library/Application Support/Google/Chrome/Default/Bookmarks")
elif system() == "Linux":
    input_filename = expanduser("~/.config/google-chrome/Default/Bookmarks")
elif system() == "Windows":
    input_filename = environ["LOCALAPPDATA"] + r"\Google\Chrome\User Data\Default\Bookmarks"
    #print("Windows")
else:
    print('System is not recognized.')
    exit(1)
    
try:
    input_file = open(input_filename, 'r', encoding='utf-8')
except IOError as e:
    if e.errno ==2:
        print("The bookmarks file could not be found in its default location ({}). ".format(e.filename) +
                "Please specify the input file manually.")
        exit(1)

In [3]:
catelog = list()
url_list = list()

def html_escape(text):
    return ''.join(html_escape_table.get(c, c) for c in text)

def html_for_node(node):
    if 'url' in node:
        return html_for_url_node(node)
    elif 'children' in node:
        return html_for_parent_node(node)
    else:
        return ''

def html_for_url_node(node):
    if not match("javascript:", node['url']):
        url_list.append(node['url'])
        return '- [{}]({})\n'.format(node['name'], node['url'])
    else:
        return ''
    
def html_for_parent_node(node):
    return '{0}\n\n{1}\n'.format(filter_catelog_name(node), ''.join([filter_name(n) for n in node['children']]))

def filter_name(n):
    if n['name'] in filter_name_list:
        return ''
    else:
        return html_for_node(n)

def filter_catelog_name(n):
    if n['name'] in filter_name_list:
        return ''
    else:
        catelog.append('- [{0}](#{0})\n'.format(n['name']))
        return '<h4 id={0}>{0}</h4>'.format(n['name'])
    
contents = loads(input_file.read())
input_file.close()

bookmark_bar = html_for_node(contents['roots']['bookmark_bar'])
#other = html_for_node(contents['roots']['other'])
catelog_str = ''.join(a for a in catelog)

In [4]:
#output_filename = 'C:/Users/lujun/Downloads/list.txt'
output_filename = 'url_list.txt'
output_file = open(output_filename, 'w', encoding='utf-8')
#output_file.write(output_file_template.format(catelog=catelog_str, bookmark_bar=bookmark_bar, other=other))
for line in url_list:
    output_file.write(line)
    output_file.write('\n')
output_file.close()